Boilerplate Connection Setup
---

In [1]:
%load_ext edx_jupyter

In [2]:
%reload_ext edx_jupyter

In [4]:
%%vertica

SELECT 'Connected'

,?column?
0,Connected


Find enrollments that are active w/in a 2 week retrospective experiment period
---

In [30]:
import datetime

start = datetime.datetime(2017, 5, 23, 10)
end = start + datetime.timedelta(days=14)

Cache the courses and enrollments for the experiment
---

In [31]:
%%vertica (start_date=start, end_date=end)
DROP TABLE IF EXISTS ex_courses;

CREATE TEMPORARY TABLE ex_courses
ON COMMIT PRESERVE ROWS
AS SELECT
    cm.course_id,
    course_start_date,
    course_seat_price
FROM
    business_intelligence.course_master AS cm
LEFT JOIN
        production.d_program_course AS pc
    ON pc.course_id = cm.course_id
WHERE
        (content_language is NULL OR content_language = 'en-us')
    AND :end_date < course_verification_end_date
    AND course_start_date < :start_date
    AND NOT is_WL

,$/enrollment,VTR (%),purchased,# enrollments


In [32]:
%vertica SELECT COUNT(*) FROM ex_courses

,COUNT
0,412


In [33]:
%%vertica (start_date=start, end_date=end)

DROP TABLE IF EXISTS ex_enrollments;

CREATE TEMPORARY TABLE ex_enrollments
ON COMMIT PRESERVE ROWS
AS SELECT
    act.*,
    uc.first_verified_enrollment_time
FROM
(
    SELECT
        act.user_id,
        act.course_id
    FROM
        business_intelligence.activity_engagement_user_daily AS act
    JOIN
            ex_courses AS crs
        ON act.course_id = crs.course_id
    WHERE
            act.is_active
        AND act.date BETWEEN :start_date AND :end_date
    GROUP BY 1, 2
) AS act
JOIN
        production.d_user_course AS uc
    ON act.course_id = uc.course_id
    AND act.user_id = uc.user_id
    AND (
            uc.first_verified_enrollment_time > :start_date
        OR uc.first_verified_enrollment_time IS NULL
    )

,COUNT


In [34]:
%vertica select count(*) / 4 from ex_enrollments

,?column?
0,86711.750000000000000000


Compute the $/enrollment, VTR and sample count
---

In [35]:
%%vertica (start_date=start, end_date=end)

SELECT
    AVG(value) AS '$/enrollment',
    COUNT(verified)/COUNT(*)*100 AS 'VTR (%)',
    COUNT(verified) AS purchased,
    COUNT(*) AS '# enrollments'
FROM
(
    SELECT
        enr.user_id,
        SUM(
            CASE
                WHEN enr.first_verified_enrollment_time <= :end_date THEN crs.course_seat_price
                ELSE 0
            END
        ) AS value,
        MIN(enr.first_verified_enrollment_time) <= :end_date AS verified
    FROM
        ex_enrollments AS enr
    JOIN
            ex_courses AS crs
        ON crs.course_id = enr.course_id
    GROUP BY 1
) AS enr
    

,$/enrollment,VTR (%),purchased,# enrollments
0,1.600332,2.686530469815690100,6775,252184
